In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:13_test_acc:0.896058.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
train_label_path = data_path + "train_labels.csv" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
BATCH_SIZE = 100
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
LEAVE_BIT_NUMBER_DICT = {10000:'1w',20000:'2w'}

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
validset = validset.iloc[np.argwhere(validset['labels'].values == 1).squeeze(),:]

In [6]:
class ExeDataset(Dataset):
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
def mp_func(i,inpu,te,gr):
    cum_grad = 0
    for j,(inp,g,t) in enumerate(zip(inpu,gr,te)):
        if inp != 0: continue
        max_idx = np.argmin(g).tolist()
        org_max_idx = np.argmax(t).tolist()
        cum_grad += np.mean(g)
        if g[max_idx] > 0: continue
        te[j][org_max_idx] = 0
        te[j][max_idx] = 1
    return [i,te,cum_grad]

In [10]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)

    def forward(self, input_, loss_fn):
        temp = F.one_hot(input_,num_classes=257).float()
        temp.requires_grad = True
        temp.retain_grad()
        batch_acc = []
        batch_grad = []
        for _ in range(9):
            x = temp @ self.embedding.weight
            x = torch.transpose(x, -1, -2)

            x_conv_1 = self.conv_layer_1(x[:,:4,:])
            x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

            x = x_conv_1*x_conv_2
            del x_conv_1,x_conv_2
            x = self.pool_layer_2(x).squeeze()
            
            x = self.fc_layer_3(x)
            x = self.fc_layer_4(x)
            
            acc = torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean()
            batch_acc.append(acc.tolist())
            
            mislead_labels = torch.cat((torch.ones(len(x)).reshape(-1,1),torch.zeros(len(x)).reshape(-1,1)),dim=1).cuda()
            loss = loss_fn(x,mislead_labels)
            loss.backward()
            
            data = [(i,inpu,te,gr) for i,(inpu,te,gr) in enumerate(zip(
                input_.detach().cpu().numpy(),
                temp.detach().cpu().numpy(),
                temp.grad.detach().cpu().numpy()
            ))]
            with mp.Pool(processes=30 if len(data) > 30 else len(data)) as pool:
                results = pool.starmap(mp_func,data)
            results = sorted(results,key = lambda x: x[0])
            for i in range(len(temp)):
                temp.data[i] = torch.tensor(results[i][1], dtype=torch.float, requires_grad=True).cuda()
            batch_grad.append(np.mean([result[2] for result in results]))
            
            # temp.grad.zero_()
            
        x = temp @ self.embedding.weight
        x = torch.transpose(x, -1, -2)

        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()

        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x.detach().cpu().numpy(),batch_acc,batch_grad

In [11]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [12]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [13]:
model.eval()
acc = []
preds = []
labels = []
total_batch_acc = []
total_batch_grad = []
bar = tqdm(validloader)
for step, (batch_data,batch_label) in enumerate(bar):
    batch_data = batch_data.cuda() if CUDA else batch_data
    batch_label = batch_label.cuda() if CUDA else batch_label
    batch_label = batch_label.squeeze()
    temp = torch.zeros((len(batch_label),2))
    for idx,target in enumerate(batch_label.squeeze()): temp[idx,target] = 1
    temp_label = temp.cuda() if CUDA else temp

    pred,batch_acc,batch_grad = model(batch_data,ce_loss)
    
    pred = np.argmax(pred,1)
    batch_label = batch_label.cpu().data.numpy()
    
    preds.extend(pred.tolist())
    labels.extend(batch_label.tolist())
    
    temp_acc = (batch_label == pred).mean()
    acc.append(temp_acc)
    total_batch_acc.append(batch_acc + [temp_acc])
    total_batch_grad.append([0] + batch_grad)
    total_batch_acc_str = '[' + ' '.join(map(lambda x: '%.7f'%x,np.mean(total_batch_acc,axis=0))) + ']'
    total_batch_grad_str = '[' + ' '.join(map(lambda x: '%.7f'%x,np.mean(total_batch_grad,axis=0))) + ']'
    print(total_batch_acc_str)
    print(total_batch_grad_str)
    print(f"test：{temp_acc:4f}, test mean: {np.mean(acc):4f}")

  0%|          | 1/600 [04:57<49:34:19, 297.93s/it]

[0.8700000 0.1200000 0.0700000 0.0500000 0.0400000 0.0400000 0.0400000 0.0400000 0.0400000 0.0400000]
[0.0000000 -0.0011380 -0.0002627 0.0011017 0.0010281 0.0010102 0.0009439 0.0009004 0.0008609 0.0008125]
test：0.040000, test mean: 0.040000


  0%|          | 2/600 [09:56<49:33:39, 298.36s/it]

[0.8650000 0.1100000 0.0550000 0.0450000 0.0400000 0.0400000 0.0350000 0.0350000 0.0350000 0.0350000]
[0.0000000 -0.0010860 -0.0004346 0.0004730 0.0004244 0.0004062 0.0003666 0.0003386 0.0003127 0.0002827]
test：0.030000, test mean: 0.035000


  0%|          | 3/600 [14:52<49:18:09, 297.30s/it]

...

 13%|█▎        | 77/600 [6:57:21<65:42:54, 452.34s/it]

[0.8871428 0.1638961 0.0966234 0.0797403 0.0759740 0.0744156 0.0720779 0.0709091 0.0706493 0.0698701]
[0.0000000 -0.0018891 -0.0012344 -0.0008028 -0.0008593 -0.0008971 -0.0009375 -0.0009744 -0.0010127 -0.0010503]
test：0.070000, test mean: 0.069870


 13%|█▎        | 78/600 [7:04:38<47:21:50, 326.65s/it]

[0.8870513 0.1628205 0.0957692 0.0789744 0.0751282 0.0735897 0.0712820 0.0701282 0.0698718 0.0691026]
[0.0000000 -0.0018818 -0.0012342 -0.0008008 -0.0008580 -0.0008953 -0.0009352 -0.0009716 -0.0010095 -0.0010466]
test：0.010000, test mean: 0.069103


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 6.
Original Traceback (most recent call last):
  File "/workdir/security/home/junjiehuang2468/.local/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/workdir/security/home/junjiehuang2468/.local/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/workdir/security/home/junjiehuang2468/.local/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-6-3d62378c0f3c>", line 12, in __getitem__
    with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
FileNotFoundError: [Errno 2] No such file or directory: '/workdir/security/home/junjiehuang2468/paper/data/ember2018/malwares/69c7d2f0ba60104816b0fb87d5cf32d